In [1]:
import time, random

import numpy as np

In [12]:
class Game:
    # skeleton structure for game (in this case single palyer)
    def __init__(self, N) -> None:
        # goal is to find most efficient route through A (always moving rightward) to reach highest value in last column
        # position is terminal if last col is reached
        self.N = N
        self.A = np.zeros((N, N))
        self.A[:, -1] = np.random.randint(0, high=N, size=N)
    
    def move(self, position, step):
        "step in [-1, 0, 1]"
        assert step in self.all_moves(), f'step {step} not allowed'
        row, col  = position
        new_row, new_col = min(self.N, max(0, row+step)), col+1
        terminal = new_col == self.N - 1
        return (new_row, new_col), terminal
    
    def all_moves(self):
        return [-1, 0, 1]
    
    def value(self, position):
        row, col  = position
        assert row in range(self.N), f'row {row} not allowed'
        assert col in range(self.N), f'column {col} not allowed'
        return self.A[row, col]        

In [19]:
class MCTS:
    def __init__(self, game, T=1, c=2):
        "T = max 'think' time"
        self.game = game
        self.T = T
        self.c = c

        self.visits = {}
        self.values = {}
    
    def run(self, root):
        self.tree = {}
        start_time = time.time()
        while time.time() - start_time < self.T:
            leaf_node, trace = self.select(root)
            children = self.expand(leaf_node)
            next_child, _ = children[0] # always start with first child from new children
            value = self.rollout(next_child)
            self.backprop(value, trace)
        root_vals = [(self.values[child]/self.visits[child], move) for child, move in self.tree[root]]
        _, move = sorted(root_vals)[-1] # pick move that leads to child with highest value/visit
    
    def select(self, node):
        trace = []
        while node in self.tree: # if not in tree -> expand (i.e. is leaf node)
            trace.append(node)
            N = self.visits[node]
            uct = [(self.c * np.sqrt(np.log(N)/self.visits[child])) for child, _ in self.tree[node]]
            _, node = max(uct)
        return node, trace

    def expand(self, node) -> None:
        children = [(self.game.move(node, move)[0], move) for move in self.game.all_moves()]
        self.tree[node] = children
        for child, _ in children:
            self.visits[child], self.values[child] = 0, 0
        return children

    def _rollout_policy(self, position):
        return random.choice(self.game.all_moves())

    def rollout(self, position) -> float:
        terminal = False
        while not terminal:
            move = self._rollout_policy(position)
            position, terminal = self.game.move(position, move)
        return self.game.value(position)

    def backprop(self, value, trace) -> None:
        for node in trace:
            self.visits[node] += 1
            self.values[node] += value


game = Game(10)
mcts = MCTS(game)
move = mcts.run((0, 0))
print(move)

TypeError: rollout() missing 1 required positional argument: 'position'

In [84]:
game = Game(10)
mcts = MCTS(game)
mcts.run((0, 0))

TypeError: can only concatenate tuple (not "int") to tuple